# Optuna lightGBM

In [1]:
import pandas as pd
import numpy as np

import lightgbm as lgb # conda install -c conda-forge lightgbm
import sklearn.metrics
from sklearn.model_selection import train_test_split

import optuna

Duplicate key in file WindowsPath('C:/Users/iiats/Anaconda3/envs/Python37-gpu/lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 250 ('font.family:  IPAexGothic')


In [2]:
df = pd.read_csv("./data/mobile_train.csv")
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [3]:
X = df.drop("price_range", axis=1).values
y = df.price_range.values

In [4]:
X

array([[8.420e+02, 0.000e+00, 2.200e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [1.021e+03, 1.000e+00, 5.000e-01, ..., 1.000e+00, 1.000e+00,
        0.000e+00],
       [5.630e+02, 1.000e+00, 5.000e-01, ..., 1.000e+00, 1.000e+00,
        0.000e+00],
       ...,
       [1.911e+03, 0.000e+00, 9.000e-01, ..., 1.000e+00, 1.000e+00,
        0.000e+00],
       [1.512e+03, 0.000e+00, 9.000e-01, ..., 1.000e+00, 1.000e+00,
        1.000e+00],
       [5.100e+02, 1.000e+00, 2.000e+00, ..., 1.000e+00, 1.000e+00,
        1.000e+00]])

In [5]:
y

array([1, 2, 2, ..., 3, 0, 3], dtype=int64)

In [6]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25)
dtrain = lgb.Dataset(X_train, label=y_train)

## lightGBM default

In [7]:
# default parameters
param = {
#         'objective': 'multiclass',
#         'metric': 'multi_logloss',
#         'num_class': 4,
        "verbosity": -1,
        "boosting_type": "gbdt"
}

In [8]:
%%time
gbm = lgb.train(param, dtrain)
preds = gbm.predict(X_val)
pred_labels = np.rint(preds)
accuracy = sklearn.metrics.accuracy_score(y_val, pred_labels)
print(accuracy)

0.91
Wall time: 78 ms


## Optuna

In [9]:
# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).

def objective(trial):
    train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.25)
    dtrain = lgb.Dataset(train_x, label=train_y)

    param = {
#         'objective': 'multiclass',
#         'metric': 'multi_logloss',
#         'num_class': 4,
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    gbm = lgb.train(param, dtrain)
    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
    return accuracy

In [10]:
%%time
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

[I 2020-12-11 18:57:15,687] A new study created in memory with name: no-name-30e71b80-485b-47c5-a0ad-e366fe33f487
[I 2020-12-11 18:57:15,718] Trial 0 finished with value: 0.922 and parameters: {'lambda_l1': 9.287297207237976e-07, 'lambda_l2': 0.02647728297673534, 'num_leaves': 51, 'feature_fraction': 0.5037118924817646, 'bagging_fraction': 0.5042874036409124, 'bagging_freq': 1, 'min_child_samples': 96}. Best is trial 0 with value: 0.922.
[I 2020-12-11 18:57:15,747] Trial 1 finished with value: 0.902 and parameters: {'lambda_l1': 5.838430748682926e-07, 'lambda_l2': 0.0002233623494249174, 'num_leaves': 105, 'feature_fraction': 0.6245687338422086, 'bagging_fraction': 0.40502834792323433, 'bagging_freq': 4, 'min_child_samples': 74}. Best is trial 0 with value: 0.922.
[I 2020-12-11 18:57:15,782] Trial 2 finished with value: 0.918 and parameters: {'lambda_l1': 7.759177570293638e-07, 'lambda_l2': 5.018350264087742e-08, 'num_leaves': 121, 'feature_fraction': 0.7865550659847884, 'bagging_fracti

[I 2020-12-11 18:57:17,364] Trial 24 finished with value: 0.928 and parameters: {'lambda_l1': 0.0013800294195102098, 'lambda_l2': 1.4159970642554e-07, 'num_leaves': 63, 'feature_fraction': 0.9382256455753825, 'bagging_fraction': 0.4591348902991177, 'bagging_freq': 4, 'min_child_samples': 60}. Best is trial 11 with value: 0.932.
[I 2020-12-11 18:57:17,430] Trial 25 finished with value: 0.92 and parameters: {'lambda_l1': 0.11942917085448539, 'lambda_l2': 1.428157087928776e-07, 'num_leaves': 145, 'feature_fraction': 0.5766702224644067, 'bagging_fraction': 0.585605575319774, 'bagging_freq': 5, 'min_child_samples': 48}. Best is trial 11 with value: 0.932.
[I 2020-12-11 18:57:17,494] Trial 26 finished with value: 0.912 and parameters: {'lambda_l1': 2.332777220665419e-05, 'lambda_l2': 8.848535017712422e-06, 'num_leaves': 114, 'feature_fraction': 0.7813736697415695, 'bagging_fraction': 0.7906445404236218, 'bagging_freq': 2, 'min_child_samples': 84}. Best is trial 11 with value: 0.932.
[I 2020-

[I 2020-12-11 18:57:18,808] Trial 48 finished with value: 0.898 and parameters: {'lambda_l1': 0.2554080639736268, 'lambda_l2': 2.7919281238849725e-05, 'num_leaves': 104, 'feature_fraction': 0.9066151158793998, 'bagging_fraction': 0.657018402599046, 'bagging_freq': 6, 'min_child_samples': 43}. Best is trial 11 with value: 0.932.
[I 2020-12-11 18:57:18,862] Trial 49 finished with value: 0.922 and parameters: {'lambda_l1': 0.02221178739265528, 'lambda_l2': 0.00035983642529720017, 'num_leaves': 53, 'feature_fraction': 0.6573442366983736, 'bagging_fraction': 0.5347225218458196, 'bagging_freq': 4, 'min_child_samples': 63}. Best is trial 11 with value: 0.932.
[I 2020-12-11 18:57:18,927] Trial 50 finished with value: 0.912 and parameters: {'lambda_l1': 1.0201709131709262, 'lambda_l2': 0.002025787053820637, 'num_leaves': 118, 'feature_fraction': 0.8794987302576899, 'bagging_fraction': 0.7087864873891342, 'bagging_freq': 3, 'min_child_samples': 58}. Best is trial 11 with value: 0.932.
[I 2020-12

[I 2020-12-11 18:57:20,499] Trial 72 finished with value: 0.914 and parameters: {'lambda_l1': 0.0022196371074062224, 'lambda_l2': 6.092184594163266e-08, 'num_leaves': 67, 'feature_fraction': 0.9384245772044325, 'bagging_fraction': 0.43814336880979404, 'bagging_freq': 4, 'min_child_samples': 34}. Best is trial 11 with value: 0.932.
[I 2020-12-11 18:57:20,574] Trial 73 finished with value: 0.918 and parameters: {'lambda_l1': 0.0316548305584723, 'lambda_l2': 2.0946704491747946e-08, 'num_leaves': 108, 'feature_fraction': 0.8834487079509623, 'bagging_fraction': 0.4656767858889254, 'bagging_freq': 5, 'min_child_samples': 31}. Best is trial 11 with value: 0.932.
[I 2020-12-11 18:57:20,640] Trial 74 finished with value: 0.912 and parameters: {'lambda_l1': 0.0006709165772449351, 'lambda_l2': 4.4327532634333324e-06, 'num_leaves': 29, 'feature_fraction': 0.9653817788189258, 'bagging_fraction': 0.48178463558987195, 'bagging_freq': 4, 'min_child_samples': 61}. Best is trial 11 with value: 0.932.
[I

[I 2020-12-11 18:57:22,169] Trial 96 finished with value: 0.902 and parameters: {'lambda_l1': 0.6189939266407131, 'lambda_l2': 0.0077971473089060985, 'num_leaves': 44, 'feature_fraction': 0.6273610822577381, 'bagging_fraction': 0.6566879339363456, 'bagging_freq': 6, 'min_child_samples': 85}. Best is trial 11 with value: 0.932.
[I 2020-12-11 18:57:22,224] Trial 97 finished with value: 0.926 and parameters: {'lambda_l1': 0.004899231117934567, 'lambda_l2': 0.001442843674571749, 'num_leaves': 113, 'feature_fraction': 0.9746996489175158, 'bagging_fraction': 0.5873739586124522, 'bagging_freq': 4, 'min_child_samples': 93}. Best is trial 11 with value: 0.932.
[I 2020-12-11 18:57:22,274] Trial 98 finished with value: 0.934 and parameters: {'lambda_l1': 0.034540109130658, 'lambda_l2': 0.0017201735971026956, 'num_leaves': 33, 'feature_fraction': 0.5249595047423804, 'bagging_fraction': 0.5428073560901951, 'bagging_freq': 6, 'min_child_samples': 97}. Best is trial 98 with value: 0.934.
[I 2020-12-1

Wall time: 6.64 s


In [11]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 100
Best trial:
  Value: 0.934
  Params: 
    lambda_l1: 0.034540109130658
    lambda_l2: 0.0017201735971026956
    num_leaves: 33
    feature_fraction: 0.5249595047423804
    bagging_fraction: 0.5428073560901951
    bagging_freq: 6
    min_child_samples: 97


In [12]:
trial.params

{'lambda_l1': 0.034540109130658,
 'lambda_l2': 0.0017201735971026956,
 'num_leaves': 33,
 'feature_fraction': 0.5249595047423804,
 'bagging_fraction': 0.5428073560901951,
 'bagging_freq': 6,
 'min_child_samples': 97}

In [13]:
%%time
gbm = lgb.train(trial.params, dtrain)
preds = gbm.predict(X_val)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000321 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1360
[LightGBM] [Info] Number of data points in the train set: 1500, number of used features: 20
[LightGBM] [Info] Start training from score 1.486667
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

In [14]:
pred_labels = np.rint(preds)
pred_labels

array([ 1.,  1.,  1.,  3.,  1.,  2.,  2.,  3.,  2.,  3., -0.,  1.,  1.,
       -0.,  3.,  3.,  2., -0.,  0.,  3., -0., -0.,  3.,  0.,  1.,  2.,
       -0.,  3.,  3., -0.,  0.,  0.,  2.,  2.,  1.,  2., -0.,  3.,  2.,
        0.,  1.,  0.,  1.,  1.,  3., -0.,  2.,  1.,  3.,  0.,  1.,  2.,
        0., -0.,  3.,  3.,  3.,  3.,  3.,  1.,  2.,  1.,  2.,  3.,  2.,
       -0.,  0.,  3.,  3.,  3.,  2., -0.,  3., -0.,  3.,  2.,  0., -0.,
        3.,  1.,  3., -0.,  1.,  0.,  2.,  3.,  0.,  1.,  3.,  0.,  3.,
        3.,  2.,  2.,  3.,  2.,  3.,  3.,  2.,  0.,  2.,  0.,  1.,  2.,
        2.,  0.,  1.,  2.,  3.,  0.,  1.,  2.,  2.,  3.,  2.,  3.,  3.,
        0.,  3.,  3.,  0., -0.,  2.,  2.,  3.,  1.,  0.,  1.,  2., -0.,
        0.,  1.,  2.,  1.,  1.,  1.,  3.,  2.,  3.,  3., -0.,  1.,  2.,
        1., -0.,  2.,  0.,  0.,  1.,  0.,  2.,  1.,  2.,  2.,  1.,  2.,
        3.,  3.,  2.,  0.,  0.,  3.,  1.,  0.,  2.,  3.,  2., -0.,  2.,
        0.,  1.,  2.,  1.,  3.,  1.,  2.,  2.,  0.,  1.,  2.,  2

In [15]:
accuracy = sklearn.metrics.accuracy_score(y_val, pred_labels)
accuracy

0.904

## Optuna lightGBM Tuner

In [16]:
import optuna.integration.lightgbm as lgb_o

In [17]:
# LightGBM用のデータセットに変換
train = lgb_o.Dataset(X_train, y_train)
val = lgb_o.Dataset(X_val, y_val)

In [18]:
%%time
# ハイパーパラメータサーチ&モデル構築
params = {'objective': 'multiclass',
          'metric': 'multi_logloss',
          'num_class': 4,
          'random_seed':0} 

gbm_o = lgb_o.train(params,
                    train,
                    valid_sets=val,
                    early_stopping_rounds=200,
                    verbose_eval=10000,)

[I 2020-12-11 18:57:22,472] A new study created in memory with name: no-name-8cc52d11-76fd-4615-a2ca-57a74db44622
feature_fraction, val_score: inf:   0%|                                                          | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000364 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1360
[LightGBM] [Info] Number of data points in the train set: 1500, number of used features: 20
[LightGBM] [Info] Start training from score -1.405137
[LightGBM] [Info] Start training from score -1.324259
[LightGBM] [Info] Start training from score -1.405137
[LightGBM] [Info] Start training from score -1.413323
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 200 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

feature_fraction, val_score: 0.188785:  14%|######4                                      | 1/7 [00:00<00:04,  1.26it/s]

No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

...

Early stopping, best iteration is:
[685]	valid_0's multi_logloss: 0.158519
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000371 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1360
[LightGBM] [Info] Number of data points in the train set: 1500, number of used features: 20
[LightGBM] [Info] Start training from score -1.405137
[LightGBM] [Info] Start training from score -1.324259
[LightGBM] [Info] Start training from score -1.405137
[LightGBM] [Info] Start training from score -1.413323
Training until validation scores don't improve for 200 rounds


min_data_in_leaf, val_score: 0.144814:  80%|####################################         | 4/5 [00:02<00:00,  1.79it/s]

Early stopping, best iteration is:
[688]	valid_0's multi_logloss: 0.152995
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000402 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1360
[LightGBM] [Info] Number of data points in the train set: 1500, number of used features: 20
[LightGBM] [Info] Start training from score -1.405137
[LightGBM] [Info] Start training from score -1.324259
[LightGBM] [Info] Start training from score -1.405137
[LightGBM] [Info] Start training from score -1.413323
Training until validation scores don't improve for 200 rounds


min_data_in_leaf, val_score: 0.144814: 100%|#############################################| 5/5 [00:02<00:00,  1.78it/s]

Early stopping, best iteration is:
[688]	valid_0's multi_logloss: 0.154041
Wall time: 50.7 s


In [19]:
# ベストパラメータの取得
best_params = gbm_o.params
print("  Params: ")
for key, value in best_params.items():
    print("    {}: {}".format(key, value))

  Params: 
    objective: multiclass
    metric: multi_logloss
    num_class: 4
    random_seed: 0
    feature_pre_filter: False
    lambda_l1: 1.567322513712503e-07
    lambda_l2: 3.1879000955360497e-05
    num_leaves: 3
    feature_fraction: 1.0
    bagging_fraction: 0.6935435173258034
    bagging_freq: 7
    min_child_samples: 20
    num_iterations: 1000
    early_stopping_round: 200


In [20]:
# 調整後モデルで予測の実行
y_trainval_pred = gbm_o.predict(X_train,num_iteration=gbm_o.best_iteration)
preds = gbm_o.predict(X_val,num_iteration=gbm_o.best_iteration)

In [21]:
preds = np.rint(preds)
preds

array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [22]:
len(preds)

500

In [23]:
preds[1][1]

1.0

In [24]:
# 実験
pred_labels = []
if preds[1][1]==1:
    pred_labels.append(1)
pred_labels

[1]

In [25]:
pred_labels = []

for i in range(len(preds)):
    if preds[i][0]==1:
        pred_labels.append(0)
    elif preds[i][1]==1:
        pred_labels.append(1)
    elif preds[i][2]==1:
        pred_labels.append(2)
    else:
        pred_labels.append(3)

In [26]:
pred_labels = np.array(pred_labels)
pred_labels

array([1, 1, 2, 3, 1, 2, 3, 3, 2, 3, 0, 1, 1, 0, 3, 3, 2, 0, 0, 3, 0, 0,
       3, 0, 0, 2, 0, 3, 3, 0, 0, 0, 2, 2, 1, 2, 0, 3, 2, 0, 1, 0, 1, 2,
       3, 0, 2, 0, 3, 0, 1, 3, 0, 0, 3, 3, 3, 3, 3, 1, 2, 1, 2, 3, 2, 0,
       0, 3, 3, 3, 2, 0, 3, 0, 3, 2, 0, 0, 2, 1, 3, 0, 1, 0, 2, 3, 0, 1,
       3, 0, 3, 3, 2, 2, 3, 2, 3, 3, 2, 0, 2, 0, 1, 2, 2, 0, 1, 2, 3, 0,
       0, 2, 2, 3, 2, 3, 3, 0, 3, 3, 0, 0, 2, 2, 3, 0, 0, 1, 2, 0, 0, 1,
       2, 1, 1, 1, 3, 2, 3, 3, 0, 1, 2, 1, 0, 2, 0, 0, 1, 0, 1, 1, 2, 1,
       1, 2, 3, 3, 2, 0, 0, 3, 1, 0, 2, 2, 2, 0, 2, 0, 1, 2, 1, 3, 1, 2,
       2, 0, 1, 2, 3, 0, 1, 3, 1, 2, 0, 2, 3, 3, 0, 3, 2, 2, 2, 3, 0, 0,
       0, 1, 2, 1, 3, 1, 0, 0, 3, 3, 0, 1, 3, 1, 2, 1, 1, 0, 1, 2, 0, 3,
       3, 3, 1, 2, 2, 0, 3, 2, 0, 2, 3, 0, 0, 3, 3, 2, 2, 0, 0, 0, 1, 2,
       1, 0, 2, 0, 2, 2, 2, 3, 3, 2, 3, 1, 1, 1, 0, 1, 3, 1, 1, 1, 0, 1,
       1, 0, 2, 0, 0, 3, 0, 1, 3, 2, 3, 1, 2, 0, 1, 3, 1, 2, 1, 2, 3, 0,
       0, 0, 2, 3, 3, 0, 0, 3, 3, 1, 2, 2, 3, 2, 3,

In [27]:
len(pred_labels)

500

In [28]:
y_val

array([1, 1, 2, 3, 1, 2, 2, 3, 2, 3, 0, 1, 1, 0, 3, 3, 2, 0, 0, 3, 0, 0,
       3, 0, 0, 2, 0, 3, 3, 0, 0, 0, 2, 2, 1, 2, 0, 3, 2, 0, 1, 0, 1, 2,
       3, 0, 2, 0, 3, 0, 2, 3, 0, 0, 3, 3, 3, 3, 3, 1, 2, 1, 2, 3, 2, 0,
       0, 3, 3, 3, 2, 0, 3, 0, 3, 2, 0, 0, 2, 1, 3, 0, 1, 0, 2, 3, 0, 1,
       3, 0, 3, 3, 2, 2, 3, 2, 3, 3, 2, 0, 2, 0, 1, 2, 2, 0, 1, 2, 3, 0,
       0, 2, 2, 3, 2, 3, 3, 0, 3, 3, 0, 0, 2, 2, 3, 0, 0, 1, 2, 0, 0, 1,
       2, 1, 1, 1, 3, 2, 3, 3, 0, 1, 2, 1, 0, 2, 0, 0, 1, 0, 1, 1, 2, 1,
       1, 2, 3, 3, 2, 0, 0, 3, 1, 0, 1, 3, 2, 0, 2, 0, 1, 2, 1, 3, 1, 2,
       2, 0, 1, 2, 2, 1, 1, 3, 1, 2, 0, 2, 3, 3, 0, 3, 2, 2, 2, 3, 0, 0,
       0, 2, 2, 1, 3, 1, 0, 0, 3, 3, 0, 1, 3, 1, 3, 1, 1, 0, 1, 2, 0, 3,
       3, 3, 1, 3, 2, 0, 3, 2, 0, 2, 3, 0, 0, 3, 3, 2, 2, 0, 0, 0, 1, 2,
       1, 0, 1, 0, 2, 2, 2, 3, 3, 2, 3, 1, 1, 0, 0, 1, 2, 1, 1, 1, 0, 1,
       1, 0, 2, 0, 0, 3, 0, 2, 3, 2, 3, 1, 2, 0, 0, 3, 1, 2, 1, 2, 3, 0,
       0, 0, 2, 3, 3, 0, 0, 3, 3, 1, 2, 2, 3, 2, 2,

In [29]:
len(y_val)

500

In [30]:
accuracy = sklearn.metrics.accuracy_score(y_val, pred_labels)
print(accuracy)

0.946


## Summary

### Random Forest
Deffault (cv=5)：0.8745 (1.38s)  
Grid search:   0.8885 (2min 50s)  
Random search: 0.8895 (1min 53s)  
Optuna:        0.8845 (1min 24s)  
Optuna(2): 0.880 (1min 12s)  
Bayesian optimization with gaussian process: KFCV: 0.905, StratifiedCV: 0.9065 (4min 15s)  
hyperopt: KFCV: 0.898, StratifiedCV: 0.898 (2min 51s) 

### lightGBM  
#### Hold out 法 (25%)
Deffault: 0.91 (78ms)  
Optuna: 0.904 (6.46s)  
Optuna lightGBM Tuner: 0.946 (50.7s)